## 6) Compare EEG references

- Some EEG features have been calculated both using an average (or CSD) reference and a zero reference
- Here, EEG features are compared between these two reference choices using multivariate distance correlations

Gordillo, da Cruz, Moreno, Garobbio, Herzog

In [ ]:
# dario.gordillolopez@epfl.ch
library('energy')
library('dplyr')

In [ ]:
# set working directory
setwd(getwd())
# csv data directory
# directory with average and csd referenced EEG features
eeg_data_dir = file.path(getwd(), "data", "csv_data")
# directory with zero referenced EEG features
z_eeg_data_dir = file.path(getwd(), "data", "csv_data", "zero_ref")

In [ ]:
# Find EEG files in .csv 
temp <- list.files(path=z_eeg_data_dir, pattern="*.csv")

# allocate memory
# multivariate distance correlations
# for young adults
z_pval_dcor_test_y <- matrix(0, length(temp), length(temp))
z_fx_dcor_test_y <- matrix(0, length(temp), length(temp))
# for older adults
z_pval_dcor_test_o <- matrix(0, length(temp), length(temp))
z_fx_dcor_test_o <- matrix(0, length(temp), length(temp))


In [ ]:
# save pairwise distance correlations between EEG features zero-referenced

for (i in 1:length(temp)){
  
  message('Processing feature ', i, ' of ', length(temp))
    
  # load EEG data
  data_EEG_1 <- read.csv(file.path(z_eeg_data_dir, temp[i]))
  # data for young adults
  y_eeg_1 <- filter(data_EEG_1, Group==1)[, 5:dim(data_EEG_1)[2]]  
  # data for older adults
  o_eeg_1 <- filter(data_EEG_1, Group==2)[, 5:dim(data_EEG_1)[2]]  

  for (j in i:length(temp)){
      
    # load EEG data
    
    if (j == i){   
        # diagonal contains distance correlation between avg/csd and zero reference
        data_EEG_2 <- read.csv(file.path(eeg_data_dir, paste(substr(temp[j], 1, nchar(temp[j])-9), '.csv', sep="")))    
    } else {        
        data_EEG_2 <- read.csv(file.path(z_eeg_data_dir, temp[j]))        
    }
    
    # data for young adults
    y_eeg_2 <- filter(data_EEG_2, Group==1)[, 5:dim(data_EEG_2)[2]] 
    # data for older adults
    o_eeg_2 <- filter(data_EEG_2, Group==2)[, 5:dim(data_EEG_2)[2]] 
    
    # multivariate distance correlation test between EEG features
    # for young adults
    multivar_ttest_y <- dcorT.test(y_eeg_1, y_eeg_2)
    z_fx_dcor_test_y[i, j] <- multivar_ttest_y$estimate
    z_pval_dcor_test_y[i, j] <- multivar_ttest_y$p.value
    
    # for older adults
    multivar_ttest_o <- dcorT.test(o_eeg_1, o_eeg_2)
    z_fx_dcor_test_o[i, j] <- multivar_ttest_o$estimate
    z_pval_dcor_test_o[i, j] <- multivar_ttest_o$p.value    
  }
}

In [ ]:
# Results for young adults
# write csv with fx magnitude
temp_ix <- gsub(".csv", "", temp)
cols_fx <- data.frame()
cols_fx <- as.data.frame(z_fx_dcor_test_y, row.names=temp_ix)
colnames(cols_fx) <- temp_ix
save_ <- file.path(getwd(), "results", "6_correlations_references_results", "6_dc_fx_y.csv")
write.csv(cols_fx, save_)

# write csv with pvalues
cols_pval <- data.frame()
cols_pval <- as.data.frame(z_pval_dcor_test_y, row.names=temp_ix)
colnames(cols_pval) <- temp_ix
save_ <- file.path(getwd(), "results", "6_correlations_references_results", "6_dc_pval_y.csv")
write.csv(cols_pval, save_)


# Results for older adults
# write csv with fx magnitude
cols_fx <- data.frame()
cols_fx <- as.data.frame(z_fx_dcor_test_o, row.names=temp_ix)
colnames(cols_fx) <- temp_ix
save_ <- file.path(getwd(),"results", "6_correlations_references_results", "6_dc_fx_o.csv")
write.csv(cols_fx, save_)

# write csv with pvalues
cols_pval <- data.frame()
cols_pval <- as.data.frame(z_pval_dcor_test_o, row.names=temp_ix)
colnames(cols_pval) <- temp_ix
save_ <- file.path(getwd(), "results", "6_correlations_references_results", "6_dc_pval_o.csv")
write.csv(cols_pval, save_)